In [ ]:
!pip install --upgrade openai


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json

# Load your dataset
csv_file = 'dataset_with_grades.csv'
data = pd.read_csv(csv_file)

# Split the dataset into training and validation sets
train, validation = train_test_split(data, test_size=0.2)

def convert_to_chat_jsonl(df, filename):
    """
    Converts a DataFrame to a JSONL file suitable for training with GPT-3.5-turbo in chat format.
    :param df: DataFrame to convert.
    :param filename: Output filename for the JSONL file.
    """
    with open(filename, 'w') as file:
        for _, row in df.iterrows():
            # Construct a chat-like interaction
            chat_interaction = {
                "messages": [
                    {"role": "system", "content": "You are a tutor assistant."},
                    {"role": "user", "content": f"What is the answer to this question: {row['Question']}"},
                    {"role": "assistant", "content": f"The answer is {row['Instructor Answer']}."},
                    {"role": "user", "content": f"My answer is: {row['Student Answer']}"},
                    {"role": "assistant", "content": f"Your answer is graded as {row['grade']}."}
                ]
            }
            file.write(json.dumps(chat_interaction) + '\n')

# Convert and save the training and validation sets
convert_to_chat_jsonl(train, 'Dataset/training_data.jsonl')
convert_to_chat_jsonl(validation, 'Dataset/validation_data.jsonl')

print("Data has been split and converted to JSONL format for chat-based model training.")


In [ ]:
from openai import OpenAI
import os

with open('/Users/shahadaleissa/NLP_project/hidden.txt') as f:
        key = f.read()
client = OpenAI(api_key=key)

client.files.create(
  file=open("Dataset/training_data.jsonl", "rb"),
  purpose="fine-tune"
)
client.files.create(
  file=open("Dataset/validation_data.jsonl", "rb"),
  purpose="fine-tune"
)

client.files.list()


In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-OeIFmrX4KtV5K4PtWTlIZ5Rn", 
  validation_file="file-RvjvE1oiYaANy9au5pbpcRPQ",
  model="gpt-3.5-turbo",
  hyperparameters={
    "n_epochs":2
  }
)

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-dUy56trxMPdEGPYA9dQQPgOu")

In [ ]:

response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8MwNH76d",
  messages= [
    {"role": "system", "content": "You are a tutor assistant."}, 
    {"role": "user", "content": "What is the answer to this question: What is a tree?"}, 
    {"role": "assistant", "content": "A tree is a hierarchical data structure defined as a collection of nodes. Nodes represent value and nodes are connected by edges. A tree has the following properties: The tree has one node called root. The tree originates from this, and hence it does not have any parent."}, 
    {"role": "user", "content": "My answer is: A data structure"}
  ]
)

print(response.choices[0].message.content)